<a href="https://colab.research.google.com/github/gmrwh92/Digital-Advertising-A-B-Test/blob/main/A_B_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facebook vs Adword A/B 테스트

# **간략 소개**

디지털 마케팅에서 효율적인 광고 캠페인을 운영하는 것은 매우 중요합니다. 광고 예산을 최적화하고 ROI(투자 대비 수익)를 높이기 위해서는 CPC, CTR, CR 등의 지표를 기반으로 한 데이터 기반 의사결정이 필수적입니다. A/B 테스트를 통해 각 광고 플랫폼이 제공하는 효율성과 전환율을 비교하고, 기업의 광고 전략을 향상시키기 위한 통찰력을 도출하는 것을 목표로 했습니다.

목표
* CTR (Click Through Rate) 비교
 - CTR은 광고나 링크가 얼마나 자주 클릭되었는지를 나타내는 수치로, 얼마나 효과적으로 사용자의 관심을 끌었는지를 평가하는 지표입니다.
* CR (Conversion Rate) 비교
  - CR (전환율)은 방문자나 광고 클릭자가 원하는 행동을 얼마나 수행했는지를 알 수 있는 평가 지표입니다. (예: 구매, 회원가입 등 실제로 고객이 된 행동)
* CPC (Cost Per AD) 비교
  - CPC는 광고 클릭당 지불하는 비용을 나타내며, 광고 캠페인의 비용 효율성을 평가하는 지표입니다.
* t-test vs z-test vs 카이검정
  - A/B 테스트에서 가설을 검정할 때 다양한 통계적 방법이 있는데, t-test, z-test, 그리고 카이제곱 검정이 가장 널리 사용됩니다. 이들의 차이를 알아보겠습니다.
  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from scipy.stats import ttest_ind
import scipy.stats as stats
from statsmodels.stats.weightstats import ztest
import numpy as np

In [3]:
## load dataset
df = pd.read_csv('/content/drive/MyDrive/A_B_testing_dataset.csv')
df

,date_of_campaign,facebook_ad_campaign,facebook_ad_views,facebook_ad_clicks,facebook_ad_conversions,facebook_cost_per_ad,facebook_ctr,facebook_conversion_rate,facebook_cost_per_click,adword_ad_campaign,adword_ad_views,adword_ad_clicks,adword_ad_conversions,adword_cost_per_ad,adword_ctr,adword_conversion_rate,adword_cost_per_click
0,2021-12-22,FACEBOOK AD,3172,62,14,141,1.95,22.58,2.27,ADWORD AD,5754,64,9,177,1.11,14.06,2.77
1,2021-12-24,FACEBOOK AD,3211,38,9,186,1.18,23.68,4.89,ADWORD AD,4954,73,6,54,1.47,8.22,0.74
2,2021-12-25,FACEBOOK AD,1936,53,15,66,2.74,28.30,1.25,ADWORD AD,4702,38,9,187,0.81,23.68,4.92
3,2021-12-25,FACEBOOK AD,1194,45,9,71,3.77,20.00,1.58,ADWORD AD,3718,58,3,161,1.56,5.17,2.78
4,2021-12-26,FACEBOOK AD,2479,44,12,174,1.77,27.27,3.95,ADWORD AD,5562,63,4,162,1.13,6.35,2.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2024-12-17,FACEBOOK AD,1154,56,8,161,4.85,14.29,2.88,ADWORD AD,4275,73,3,54,1.71,4.11,0.74
996,2024-12-18,FACEBOOK AD,1108,58,13,198,5.23,22.41,3.41,ADWORD AD,4749,55,5,116,1.16,9.09,2.11
997,2024-12-18,FACEBOOK AD,1866,70,19,97,3.75,27.14,1.39,ADWORD AD,4582,36,4,149,0.79,11.11,4.14
998,2024-12-20,FACEBOOK AD,1279,59,19,171,4.61,32.20,2.90,ADWORD AD,4877,66,6,65,1.35,9.09,0.98


In [4]:
## 필요한 column만 추출 ctr -> click-through-rate (광고 노출수 대비 클릭수), conversion_rate (광고 클릭수 대비 실제 고객으로 변환된수). cost_per_click(광고 클릭당 지출 비용)
facebook_data = df[['facebook_ctr', 'facebook_conversion_rate', 'facebook_cost_per_click']]
adwords_data = df[['adword_ctr', 'adword_conversion_rate', 'adword_cost_per_click']]

In [5]:
# Facebook Ads 평균 지표 계산
facebook_means = facebook_data.mean()

# AdWords 평균 지표 계산
adwords_means = adwords_data.mean()

# 결과 출력
print("Facebook Ads 평균 지표:")
print(facebook_means)
print("\nAdWords 평균 지표:")
print(adwords_means)

Facebook Ads 평균 지표:
facebook_ctr                 2.28316
facebook_conversion_rate    32.69608
facebook_cost_per_click      4.31050
dtype: float64

AdWords 평균 지표:
adword_ctr                 1.28095
adword_conversion_rate    10.80751
adword_cost_per_click      2.26554
dtype: float64


간단한 통계 지표로 확인한 결과, 페이스북과 AdWords의 평균 지표 간에 차이가 유의미한 것으로 나타났습니다. A/B 테스트를 통해 이 차이가 실제로 의미 있는지 확인하고자 합니다. t-test를 수행하기 전에, 두 표본의 분산이 동일한지를 확인하고자 합니다. 이는 t-test의 전제 조건 중 하나로, 두 표본의 분산이 비슷해야 신뢰할 수 있는 결과를 도출할 수 있기 때문입니다.

In [6]:
facebook_ctr = df['facebook_ctr']
adwords_ctr = df['adword_ctr']

levene_stat, levene_p_value = stats.levene(facebook_ctr, adwords_ctr)

# 결과 출력
print(f"Levene's Test Statistic: {levene_stat}, p-value: {levene_p_value}")

# p-value가 0.05보다 작으면 분산이 다르다는 결론
if levene_p_value < 0.05:
    print("분산이 동일하지 않다고 할 수 있습니다.")
else:
    print("분산이 동일하다고 할 수 있습니다.")

Levene's Test Statistic: 521.9847120721529, p-value: 7.780563414133929e-103
분산이 동일하지 않다고 할 수 있습니다.


In [7]:
facebook_cr = df['facebook_conversion_rate']
adwords_cr = df['adword_conversion_rate']

levene_stat, levene_p_value = stats.levene(facebook_cr, adwords_cr)

# 결과 출력
print(f"Levene's Test Statistic: {levene_stat}, p-value: {levene_p_value}")

# p-value가 0.05보다 작으면 분산이 다르다는 결론
if levene_p_value < 0.05:
    print("분산이 동일하지 않다고 할 수 있습니다.")
else:
    print("분산이 동일하다고 할 수 있습니다.")

Levene's Test Statistic: 458.58588999717966, p-value: 9.297580279831654e-92
분산이 동일하지 않다고 할 수 있습니다.


In [8]:
facebook_cpc = df['facebook_cost_per_click']
adwords_cpc = df['adword_cost_per_click']

levene_stat, levene_p_value = stats.levene(facebook_cpc, adwords_cpc)

# 결과 출력
print(f"Levene's Test Statistic: {levene_stat}, p-value: {levene_p_value}")

# p-value가 0.05보다 작으면 분산이 다르다는 결론
if levene_p_value < 0.05:
    print("분산이 동일하지 않다고 할 수 있습니다.")
else:
    print("분산이 동일하다고 할 수 있습니다.")

Levene's Test Statistic: 280.8445952915957, p-value: 4.391110567636265e-59
분산이 동일하지 않다고 할 수 있습니다.


 t-test는 표본 샘플이 작을때 주로 사용하는 방법. 사용하는 데이터세트의 샘플은 1000이고 Ctr, Cr, Cpc 모두 분산이 동일하지 않다라는 결론이 나왔지만 그럼에도 불구하고 기본 t-test 먼저 확인.

In [9]:
# CTR 비교
ctr_ttest = ttest_ind(facebook_data['facebook_ctr'], adwords_data['adword_ctr'])

# Conversion Rate 비교
cr_ttest = ttest_ind(facebook_data['facebook_conversion_rate'], adwords_data['adword_conversion_rate'])

# Cost per Click 비교
cpc_ttest = ttest_ind(facebook_data['facebook_cost_per_click'], adwords_data['adword_cost_per_click'])

# 결과 출력
print("CTR T-test 결과:", ctr_ttest)
print("Conversion Rate T-test 결과:", cr_ttest)
print("Cost per Click T-test 결과:", cpc_ttest)


CTR T-test 결과: TtestResult(statistic=24.914527350282636, pvalue=1.5315526443075186e-119, df=1998.0)
Conversion Rate T-test 결과: TtestResult(statistic=32.62045547744514, pvalue=1.7501387675629208e-187, df=1998.0)
Cost per Click T-test 결과: TtestResult(statistic=21.817619658526826, pvalue=7.864512933361507e-95, df=1998.0)


z-test는 표본 크기가 충분히 큰 경우(보통 30명 이상)나 모집단의 분포가 정규분포인 경우에 사용됩니다. 주로 모평균이나 모비율을 비교할 때 사용됩니다. 데이터셋의 샘플 크기가 1000이고, 분산을 알고 있기 때문에 z-test를 실행

In [10]:
## t-test와 마찬가지로 library 사용
# CTR 비교
ctr_ztest = ztest(facebook_data['facebook_ctr'], adwords_data['adword_ctr'])

# Conversion Rate 비교
cr_ztest = ztest(facebook_data['facebook_conversion_rate'], adwords_data['adword_conversion_rate'])

# Cost per Click 비교
cpc_ztest = ztest(facebook_data['facebook_cost_per_click'], adwords_data['adword_cost_per_click'])

# 결과 출력
print("CTR T-test 결과:", ctr_ztest)
print("Conversion Rate T-test 결과:", cr_ztest)
print("Cost per Click T-test 결과:", cpc_ztest)

CTR T-test 결과: (24.914527350282636, 5.178286566673302e-137)
Conversion Rate T-test 결과: (32.62045547744515, 2.103495338514956e-233)
Cost per Click T-test 결과: (21.81761965852683, 1.5786528601085756e-105)


In [11]:
wctr_ttest = ttest_ind(facebook_data['facebook_ctr'], adwords_data['adword_ctr'], equal_var=False)
wcr_ttest = ttest_ind(facebook_data['facebook_conversion_rate'], adwords_data['adword_conversion_rate'], equal_var=False)
wcpc_ttest = ttest_ind(facebook_data['facebook_cost_per_click'], adwords_data['adword_cost_per_click'], equal_var=False)

# 결과 출력
print("CTR Welch's T-test 결과:", wctr_ttest)
print("Conversion Rate Welch's T-test 결과:", wcr_ttest)
print("Cost per Click Welch's T-test 결과:", wcpc_ttest)

CTR Welch's T-test 결과: TtestResult(statistic=24.914527350282636, pvalue=4.489219631688428e-111, df=1218.5683988369776)
Conversion Rate Welch's T-test 결과: TtestResult(statistic=32.62045547744514, pvalue=7.116026506592085e-165, df=1126.4031934911668)
Cost per Click Welch's T-test 결과: TtestResult(statistic=21.81761965852683, pvalue=2.790108005850851e-90, df=1311.3536565344236)


마지막 섹션에서 실행한 Welch's T-Test는 기본 t-검정에서 두 그룹의 분산이 다르거나 데이터가 정규분포를 따르지 않는 경우에 사용하는 방법.

# **결론**
페이스북과 AdWords 광고의 CTR (클릭률), CR (전환율), CPC (클릭당 비용) 분석 결과, 두 광고 플랫폼 간에 유의미한 차이가 있다는 걸 통계적으로 확인함. 이는 두 플랫폼이 다른 성과를 보인다는 의미로, 광고 캠페인 전략을 다시 점검하고 조정할 필요가 있다는 걸 시사함. 이를 바탕으로, 광고 캠페인을 더욱 효율적으로 개선하기 위한 해결책을 제시할 수 있음
* 예시
 - 페이스북 광고의 CTR이 높기때문에 페이스북 광고가 더 많은 관심을 끌고 있다라는 결론 도출 가능. 페이스북 타겟팅 강화 혹은 AdWords의 CTR 상승 위한 개선 필요
 - 페이스북의 CR이 더 높기 때문에 페이스북의 광고 클릭 후 사용자 경험이 더 낫다라는 결론 도출 가능. 광고 문구, 디자인등 사용자 참여율에 강세를 보인만큼 어느 부분이 참여율로 전환되게 하는 요소인지 파악후 강점 강화 가능
 - 페이스북의 CPC가 더 높다는 말은 페이스북이 AdWords보다 더 많은 비용을 광고에 지출했다는 결과. 경쟁이 비효율적이라는 가능성 존재. 잠재적으로 더 많은 고객을 끌어 당긴 후 그에 대한 매출이 증가 할 수 잇는 가능성도 존재하나, 더욱 효율적인 비용대비 효과적인 광고 전략이 필요하다는 결론도 도출 가능.<br>

통계적 검정 결과에 따라, 페이스북과 AdWords의 광고 성과가 상이하다는 것을 알 수 있었습니다. 이를 바탕으로 각 광고 플랫폼의 장점과 단점을 분석하고, 광고 예산 최적화, 타겟팅 전략, 사용자 경험 개선 등을 통해 더욱 효과적인 캠페인 운영 가능. 이제 광고 캠페인 전략을 데이터 기반 의사결정을 통해 지속적으로 개선하며, ROI를 극대화할 수 있는 기회를 가질 수 있음